In [3]:
# Import libraries
using Turing
using LinearAlgebra
using Distributions
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using StatsBase
using Statistics
using Suppressor
using JLD2
using CSV
using DataFrames, DataFramesMeta
using SplitApplyCombine
using KernelFunctions
using MCMCChains
using PyCall
using PyPlot
using Printf
import PyCall.pyfunction
using Optim

include("../Utils/scale_utils.jl")
using .ScaleUtils

include("../Utils/gp_utils.jl")
using .GPUtils

# Access the matplotlib module
matplotlib = pyimport("matplotlib")
seaborn = pyimport("seaborn")
os = pyimport("os")
pyimport("scienceplots")
np = pyimport("numpy")
scipy = pyimport("scipy")
skl_model_selection = pyimport("sklearn.model_selection")
plt.style.use(["default","science","no-latex"])
using StatsPlots

seaborn.color_palette("colorblind")

PyCall.pygui(:tk)

# Set a seed for reproducibility
using Random
Random.seed!(11);

#import Pkg
#Pkg.add("PrettyTables")
FONTSIZE=20.5;

In [4]:
matplotlib_backends_pdf = pyimport("matplotlib.backends.backend_pdf")
PdfPages = matplotlib_backends_pdf.PdfPages

PyObject <class 'matplotlib.backends.backend_pdf.PdfPages'>

In [5]:
function unscale_params(params)
    
    unscaled_θ_1 = ( params[1] * (X_maxs[:vmThresh] - X_mins[:vmThresh]) ) + X_mins[:vmThresh]
    unscaled_θ_2 = ( params[2] * (X_maxs[:fricExp] - X_mins[:fricExp]) ) + X_mins[:fricExp]
    unscaled_θ_3 = ( params[3] * (X_maxs[:mu_scale] - X_mins[:mu_scale]) ) + X_mins[:mu_scale]
    unscaled_θ_4 = ( params[4] * (X_maxs[:stiff_scale] - X_mins[:stiff_scale]) ) + X_mins[:stiff_scale]
    unscaled_θ_5 = ( params[5] * (X_maxs[:gamma0] - X_mins[:gamma0]) ) + X_mins[:gamma0]
    unscaled_θ_6 = ( params[6] * (X_maxs[:melt_flux] - X_mins[:melt_flux]) ) + X_mins[:melt_flux];
    unscaled_thetas = [unscaled_θ_1
                        unscaled_θ_2
                        unscaled_θ_3
                        unscaled_θ_4
                        unscaled_θ_5
                        unscaled_θ_6];
    return unscaled_thetas
    
end

unscale_params (generic function with 1 method)

In [6]:
#All years from 2016 - 2300
all_years_no_gap = collect(range(2016, step=1, length=285))
yrs_gapped = collect(range(2030, step=15, length=19));

# Mapping those years to indicies in a list
yrs_dict = Dict{Int64, Int64}()
for (idx,yr) in enumerate(all_years_no_gap)
        yrs_dict[yr] = idx
end


In [7]:
#Loading original parameter data for scaling purposes
X_raw = CSV.read("../Data/Training_Data/Amery_Input_Parameters_Filtered.csv", DataFrame);
# 1) Grab all column‐names as Symbols
cols = Symbol.(names(X_raw))
# 2) Remove the index‐column symbol
cols = filter(c -> c != :Column1, cols)
# 3) Now call get_scaled_matrix on the remaining columns
X_scaled_t, X_scalers, X_mins, X_maxs = ScaleUtils.get_scaled_matrix(X_raw, cols);

function tuple_to_idx(x,y)
    return ( (3*x - 3) + y )
end

tuple_to_idx (generic function with 1 method)

In [10]:
#Returns means, upper, and lower quntailes for a single year's prediction from a single (R, yr_calibrated) pair
function calculate_quantiles_mean(sample, row)
    slice = sample[row, :]
    mean_data = mean(slice)
    quantile_5 = quantile(slice, 0.05)
    quantile_95 = quantile(slice, 0.95)
    
    return mean_data, quantile_5, quantile_95

end

calculate_quantiles_mean (generic function with 1 method)

In [11]:
#Function to plot the six parameter values against the learning observed for their corresponding trajectories
function plot_figure_five(calibrating_year, year_projected, pdf_saveout)
        
        fig, ax = PyPlot.subplots(nrows=2, ncols=3, figsize=(18, 11), dpi=300,
                             gridspec_kw=Dict("height_ratios"=> [1, 1], "width_ratios" => [1, 1, 1],
                                            "wspace"=> 0.175, "hspace"=> 0.25))
        #Vectors to hold all quantities of interest from the trajectories
        widths = Vector{Float64}()
        params = Dict([1 => Vector{Float64}(), 2 => Vector{Float64}() , 3 => Vector{Float64}() ,
            4 => Vector{Float64}(), 5 => Vector{Float64}(), 6 => Vector{Float64}() ])

        #Change the path to wherever your SLR Projections are stored
        SLR_projections_path = "../../BrookhavenCode/SLR_Projection_Data/Sanket_Results_Projection"
    
        #Go through each folder in the directory containing the SLR projection data
        for dir in readdir(SLR_projections_path)
            if occursin("R_", dir)
                #Grab the index of the file string containing the realization number
                n = Int(findfirst("_", dir).start)
                r = dir[n+1:end]
                # Load this specific sample of projections
                proj = JLD2.load("$(SLR_projections_path)/$(dir)/$(r)-year$(calibrating_year)pred_VAF.jld2", 
                    "sample_post_mm_ssp5" )            
                #Calculate_quantiles and width of the projections of the chosen year
                mean, lower, upper = calculate_quantiles_mean(proj, yrs_dict[year_projected])
                int_width = upper - lower
                push!(widths, int_width)
                #Load the parameters that generated this trajectories future realstic observations
                θ = JLD2.load("../Data/Future_Observation_Data/Generative_Parameters/0.8244943887658679-data/$(r)_emulator_data.jld2", "θ")
                vec_θ = vec(θ)
                θ_un = unscale_params(θ)
                #Push each parameter into its respective list
                for (idx, el) in enumerate(θ_un)
                    push!(params[idx], el)
                end

            end
        end
    
        #Plotting
        for i in 1:2
            for j in 1:3
                var_idx = tuple_to_idx(i,j)
                ax[i,j].scatter(params[var_idx], widths,label="Individual Trajectory")
                #Linear Regression line
                m, c = np.polyfit(params[var_idx], widths, 1)
                ax[i,j].plot(params[var_idx], (params[var_idx] .* m .+ c), color = "red",label="Linear regression")
                ax[i,j].set_title(title_dict[var_idx],fontsize=1.2*FONTSIZE, pad=10)    #("von Mises threshold (Pa)")        
                ax[i,j].tick_params(axis="both", which="major", labelsize=FONTSIZE*0.8)
                ax[i,j].grid(true, alpha=0.25, zorder=1)
                ax[i,j].set_xticks(tick_dict[var_idx])

            end
        end
        handles, labels = ax[1,1].get_legend_handles_labels()
        desired_indices = [1,2]
        sorted_handles = [handles[i] for i in desired_indices]
        sorted_labels = [labels[i] for i in desired_indices]
        fig.legend(sorted_handles, sorted_labels, loc="upper center", ncol=6, fontsize=FONTSIZE,
                   bbox_to_anchor=(0.512, 1), bbox_transform=PyPlot.gcf().transFigure,
                   frameon="True", framealpha=1)
        fig.supxlabel("Parameter value", fontsize=FONTSIZE*1.2)
        fig.supylabel("90% uncertainty range", fontsize=FONTSIZE*1.2)
    
        pdf_saveout.savefig(fig)
        PyPlot.close(fig)

end

plot_figure_five (generic function with 1 method)

In [14]:
title_dict = Dict([
            1 => L"\sigma_{max}", 2 => L"q" , 3 => L"C_{\mu}" ,
            4 => L"C_{\phi}", 5 => L"\gamma_0", 6 => L"\overline{m}" ]) 

tick_dict = Dict([
        1 => [110000,120000,130000,140000], 2 => [0.15,0.2,0.25,0.3], 3 => [0.8,0.9,1,1.1],
        4 => [0.85,0.9,0.95,1,1.05], 5 => [50000,150000,250000], 6 => [20,30,40,50] ]) 

#Defining the years at which calibrations take place
cal_years = collect(range(2030,step=15,length=19))
#Define the year for which you want projection uncertainties
chosen_pred_year = 2300
#Choose only the calibration years before that chosen projection year
truncated_years = filter(x -> x <= chosen_pred_year, cal_years)

#Define your plot saveout location
param_space_learning_saveout = "../Plots/Figure_Four_Plots/parameter_space_vs_slr_learning_draft_$(chosen_pred_year).pdf"
pdf_pages = PdfPages(param_space_learning_saveout)

try       
    for i in 1:length(truncated_years)
        plot_figure_five(cal_years[i], chosen_pred_year, pdf_pages)
    end     
    
finally 
    # --- Always close the PdfPages object to finalize the PDF file ---
    # This ensures the PDF is properly written to disk and not corrupted.
    pdf_pages.close()
end  
